In [ ]:
# Cell 0: IMPORTS
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops

# Importar módulos da mesma pasta
from preprocessing import converter_para_cinza, aplicar_filtro_gaussiano


In [ ]:
imagem = cv2.imread("../../dataset/Apple/3_100.jpg")

cinza = converter_para_cinza(imagem)
gaussiano = aplicar_filtro_gaussiano(cinza)

In [ ]:
# Cell 2: LOCAL BINARY PATTERN (LBP)
def calcular_lbp(imagem, P=8, R=1, metodo="uniform"):
    """
    Calcula o Local Binary Pattern (LBP) de uma imagem.
    
    Args:
        imagem: Imagem em escala de cinza
        P: Número de pontos na vizinhança circular (padrão: 8)
        R: Raio da vizinhança circular (padrão: 1)
        metodo: Método de cálculo ('default', 'ror', 'uniform', 'nri_uniform', 'var')
        
    Returns:
        tuple: (imagem_lbp, histograma_lbp)
    """
    # Calcular LBP
    lbp = local_binary_pattern(imagem, P=P, R=R, method=metodo)
    
    # Normalizar para visualização (0-255)
    lbp_norm = (lbp - lbp.min()) / (lbp.max() - lbp.min() + 1e-9)
    lbp_img = (lbp_norm * 255).astype("uint8")
    
    # Calcular histograma
    lbp_bins = np.arange(0, lbp.max() + 2)
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=lbp_bins, density=True)
    
    return lbp_img, lbp_hist

# Calcular LBP
lbp_img, lbp_hist = calcular_lbp(cinza)

print(f"LBP calculado:")
print(f"  - Imagem LBP: {lbp_img.shape}")
print(f"  - Histograma: {len(lbp_hist)} bins")
print(f"  - Primeiros 10 valores: {lbp_hist[:10]}")

LBP calculado:
  - Imagem LBP: (100, 100)
  - Histograma: 10 bins
  - Primeiros 10 valores: [0.0342 0.0612 0.0454 0.1174 0.1837 0.1693 0.064  0.0673 0.1503 0.1072]


In [ ]:
# Cell 3: GRAY LEVEL CO-OCCURRENCE MATRIX (GLCM)
def calcular_glcm(imagem, distancias=[1, 2, 3], angulos=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    """
    Calcula características de textura usando Gray Level Co-occurrence Matrix (GLCM).
    
    Args:
        imagem: Imagem em escala de cinza
        distancias: Lista de distâncias para calcular GLCM (padrão: [1, 2, 3])
        angulos: Lista de ângulos em radianos (padrão: [0, π/4, π/2, 3π/4])
        
    Returns:
        dict: Dicionário com características de textura:
            - contrast: Contraste
            - dissimilarity: Dissimilaridade
            - homogeneity: Homogeneidade
            - ASM: Angular Second Moment
            - energy: Energia
            - correlation: Correlação
    """
    # Calcular matriz GLCM
    glcm = graycomatrix(
        imagem, 
        distances=distancias, 
        angles=angulos, 
        levels=256, 
        symmetric=True, 
        normed=True
    )
    
    # Extrair propriedades
    caracteristicas = {}
    propriedades = ["contrast", "dissimilarity", "homogeneity", "ASM", "energy", "correlation"]
    
    for prop in propriedades:
        # Média de todas as distâncias e ângulos
        caracteristicas[prop] = float(graycoprops(glcm, prop).mean())
    
    return caracteristicas

# Calcular GLCM
glcm_caracteristicas = calcular_glcm(cinza)

print("Características GLCM:")
for prop, valor in glcm_caracteristicas.items():
    print(f"  - {prop}: {valor:.4f}")

Características GLCM:
  - contrast: 811.7464
  - dissimilarity: 11.5655
  - homogeneity: 0.2734
  - ASM: 0.0133
  - energy: 0.1148
  - correlation: 0.9256
